In [ ]:
%%writefile Parallel.cu
#include <stdio.h>
#include <malloc.h>
#include <cuda.h>
#define Nx 16
#define Ny 16
#define Nt 21
#define dxy 0.1
#define dt 0.01
#define GridSize 1
#define BlockSize 8
#define ThreadSize Nx*Ny/(GridSize*BlockSize)
//==============================
float funct1(int i, int j) {
  return sin(M_PI*(i*dxy + j*dxy));
}
float funct2(int i, int j) {
  return cos(M_PI*(i*dxy + j*dxy));
}
//==============================
void InputData(float *U, float *V, float *C) {
  int i,j,k;
  for (j=0; j<Ny; j++) {
    for (i=0; i<Nx; i++) {
      *(U + j*Nx + i) = funct1(i,j);
      *(V + j*Nx + i) = dt*funct2(i,j) + *(U + j*Nx + i);
    }
  }
  for (k=0; k<Nt; k++) {
    *(C+k) = 1/2;
  }
}
//===========================
__global__ void Computing(float *U, float *V, float *C){
  float left, right, bottom, top, center1, center2;
  int k, l, index, start, stop, nt_half;
  index = blockIdx.x * blockDim.x + threadIdx.x;
  start = index*ThreadSize;
  stop  = start + ThreadSize;
  nt_half = (Nt+1)/2;
  for (k=1; k<nt_half; k++) {
    for (l=start; l<stop; l++) {
      left  =  (l%Nx == 0)       ? 0 : *(U + l - 1);
      right =  (l%Nx == Nx-1)    ? 0 : *(U + l + 1);
      bottom = (l >= Nx*(Ny-1))  ? 0 : *(U + l - Nx);
      top   =  (l <= Nx-1)       ? 0 : *(U + l + Nx);
      center1 = *(V + l);
      center2 = *(U + l);
      *(U + l) = 2*center1 - center2 + ((dt * dt * (*(C + k)) * (*(C + k))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
    }
    __syncthreads();
    for (l=start; l<stop; l++) {
      left  =  (l%Nx == 0)       ? 0 : *(V + l - 1);
      right =  (l%Nx == Nx-1)    ? 0 : *(V + l + 1);
      bottom = (l >= Nx*(Ny-1))  ? 0 : *(V + l + Nx);
      top   =  (l <= Nx-1)       ? 0 : *(V + l - Nx);
      center1 = *(U + l);
      center2 = *(V + l);
      *(V + l) = 2*center1 - center2 + ((dt * dt * (*(C + k)) * (*(C + k))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
    }
    __syncthreads();
  }
}
//===========================
int main() {
  float *UCPU, *VCPU, *CCPU;
  UCPU = (float *) malloc (Nx*Ny*sizeof(float));
  VCPU = (float *) malloc (Nx*Ny*sizeof(float));
  CCPU = (float *) malloc (Nt*sizeof(float));
  InputData(UCPU, VCPU, CCPU);

  // Delare and Allocate Mem on GPU
  float *UGPU, *VGPU, *CGPU;
  cudaMalloc((void**)&UGPU ,Nx*Ny*sizeof(float));
  cudaMalloc((void**)&VGPU ,Nx*Ny*sizeof(float));
  cudaMalloc((void**)&CGPU ,Nt*sizeof(float));

  // Copy Input from CPU to GPU
  cudaMemcpy(UGPU,UCPU,Nx*Ny*sizeof(float),cudaMemcpyHostToDevice);
  cudaMemcpy(VGPU,VCPU,Nx*Ny*sizeof(float),cudaMemcpyHostToDevice);
  cudaMemcpy(CGPU,CCPU,Nt*sizeof(float),cudaMemcpyHostToDevice);

  //Define Block and Thread Structure
  dim3 dimGrid(GridSize);
  dim3 dimBlock(BlockSize);

  //Computing
  Computing<<<dimGrid,dimBlock>>>(UGPU, VGPU, CGPU);

  //Copy Output from GPU to CPU
  cudaMemcpy(UCPU, UGPU, Nx*Ny*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(VCPU, VGPU, Nx*Ny*sizeof(float), cudaMemcpyDeviceToHost);
  cudaMemcpy(CCPU, CGPU, Nt*sizeof(float), cudaMemcpyDeviceToHost);

  //Show result
  if (Nt%2 == 1) {
    for (int j=0; j<Ny; j++) {
      for (int i=0; i<Nx; i++) {
        printf("%f ", *(UCPU + j*Nx +i));
      }
    printf("\n");
    }
  }
  else {
    for (int j=0; j<Ny; j++) {
      for (int i=0; i<Nx; i++) {
        printf("%f ", *(VCPU + j*Nx +i));
      }
    printf("\n");
    }
  }

  // Free Mem on CPU and GPU
  free(UCPU); free(VCPU); free(CCPU);
  cudaFree(UGPU); cudaFree(VGPU); cudaFree(CGPU);
  return 0;
}

In [78]:
!nvcc Parallel.cu -lm

In [ ]:
!./a.out